In [ ]:
import csv
from datetime import datetime

class Room:
    """Stores information about a room and its temperatures."""
    def __init__(self, room_name, temp_now, temp_goal):
        self.room_name = room_name
        self.temp_now = temp_now
        self.temp_goal = temp_goal


class HeatingAgent:
    """Model-based reflex agent to decide heater actions."""
    def __init__(self):
        self.last_action = {}
        self.logs = []

    def read_temperature(self, room: Room):
        """Get the current temperature of the room."""
        return room.temp_now

    def decide(self, room: Room):
        """Decide whether to turn on/off heater or take no action."""
        if room.temp_now < room.temp_goal:
            action = "Heater ON"
        else:
            action = "Heater OFF"

        # Prevent repeating the same action
        if self.last_action.get(room.room_name) == action:
            action = "No change"
        else:
            self.last_action[room.room_name] = action

        return action

    def perform(self, room: Room):
        """Perform action and keep log."""
        action_taken = self.decide(room)
        log_data = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "room": room.room_name,
            "current_temp": room.temp_now,
            "target_temp": room.temp_goal,
            "action": action_taken,
            "last_action": self.last_action.get(room.room_name)
        }
        self.logs.append(log_data)
        return action_taken


# e.g
room_list = [
    Room("Living Room", 18, 22),
    Room("Bedroom", 22, 21),
    Room("Kitchen", 20, 22),
    Room("Bathroom", 24, 23)
]

agent = HeatingAgent()

# Write log file
with open("heating_agent_log.csv", mode="w", newline="") as file:
    headers = ["timestamp", "room", "current_temp", "target_temp", "action", "last_action"]
    csv_writer = csv.DictWriter(file, fieldnames=headers)
    csv_writer.writeheader()

    for rm in room_list:
        sensed = agent.read_temperature(rm)
        act = agent.perform(rm)
        print(f"[{rm.room_name}] Now = {sensed}°C | Target = {rm.temp_goal}°C → {act}")
        csv_writer.writerow(agent.logs[-1])


[Living Room] Now = 18°C | Target = 22°C → Heater ON
[Bedroom] Now = 22°C | Target = 21°C → Heater OFF
[Kitchen] Now = 20°C | Target = 22°C → Heater ON
[Bathroom] Now = 24°C | Target = 23°C → Heater OFF
